In [1]:
#######################################################################################
import hashlib
import json
import os
import time
import random
from datetime import datetime
#######################################################################################

In [2]:
# File paths
USERS_FILE = 'users.json'
QUIZZES_FILE = 'quiz_questions.json'
RESULTS_FILE = 'results.json'

In [3]:
# Load data from JSON files
def load_data(file_path):
    """
    Load data from a JSON file.

    Args:
        file_path (str): The path to the JSON file to load.

    Returns:
        dict: The data loaded from the file, or an empty dictionary if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            print(f"Successfully loaded data from {file_path}")
            return data
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {file_path}. Check JSON format.")
        return {}
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return {}

In [4]:
def save_data(file_path, data):
    """
    Saves data to a JSON file.

    Args:
        file_path (str): The path to the JSON file to save data to.
        data (dict): The data to save in the JSON file.
    """
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [5]:
# Hash password
def hash_password(password):
    """
    Hashes a password using SHA-256.

    Args:
        password (str): The password to hash.

    Returns:
        str: The hashed password in hexadecimal format.
    """
    return hashlib.sha256(password.encode()).hexdigest()

In [6]:
# Send verification code (simulate sending)
def send_verification_code(email, code):
    """
    Simulates sending a verification code to the user's email.

    Args:
        email (str): The user's email address.
        code (str): The verification code to send.
    """
    print(f"Verification code sent to {email}@gmail.com: {code}")

In [7]:
# Register a new user
def register_user(username, password, role):
    """
    Registers a new user with a username, password, and role.

    Args:
        username (str): The desired username for the new user.
        password (str): The password for the new user.
        role (str): The role of the user (admin or participant).

    Returns:
        bool: True if registration was successful, False otherwise.
    """
    users = load_data(USERS_FILE)
    if username in users:
        print("Username already exists.")
        return False

    # Generate a verification code
    verification_code = str(random.randint(100000, 999999))
    email = input("Enter your email address for verification: ")
    send_verification_code(email, verification_code)

    # Ask for the verification code
    user_input_code = input("Please enter the verification code sent to your email: ")

    if user_input_code == verification_code:
        print("Verification successful!")
        users[username] = {
            'password': hash_password(password),
            'role': role
        }
        save_data(USERS_FILE, users)
        print(f"User '{username}' registered successfully as {role}.")
        return True
    else:
        print("Invalid verification code. Registration failed.")
        return False

In [8]:
# Login user
def login_user(username, password):
    """
    Logs in a user by verifying their username and password.

    Args:
        username (str): The username of the user.
        password (str): The password of the user.

    Returns:
        str or None: The role of the user if login is successful, None otherwise.
    """
    users = load_data(USERS_FILE)
    if username in users and users[username]['password'] == hash_password(password):
        print(f"Welcome, {username}!")
        return users[username]['role']
    else:
        print("Invalid credentials.")
        return None

In [9]:
# Create a new quiz (admin only)
def create_quiz(admin_username):
    """
    Allows an admin user to create a new quiz.

    Args:
        admin_username (str): The username of the admin creating the quiz.
    """
    quizzes = load_data(QUIZZES_FILE)
    question_bank = load_data('question_bank.json')  # Assuming you have a separate question bank JSON
    quiz_id = f"quiz_{int(time.time())}"  # Unique quiz ID based on current timestamp

    # Prompt for quiz details
    topic = input("Enter the quiz topic: ")
    password = input("Set a password for this quiz: ")
    num_questions = int(input("How many questions do you want in this quiz? "))
    
    # Validate number of questions
    if num_questions > len(question_bank):
        print(f"You can only select up to {len(question_bank)} questions from the question bank.")
        return

    # Randomly select questions from the question bank
    selected_questions = random.sample(question_bank, num_questions)

    # Structure the quiz data
    quizzes[quiz_id] = {
        "admin": admin_username,
        "topic": topic,
        "password": password,
        "questions": selected_questions,
        "created_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Save the quiz data to quiz_questions.json
    save_data(QUIZZES_FILE, quizzes)
    print(f"Quiz '{quiz_id}' created successfully.")

In [10]:
# Take a quiz
def take_quiz(username):
    """
    Facilitates a participant's quiz-taking experience.

    Args:
        username (str): The username of the participant.
    """
    quizzes = load_data(QUIZZES_FILE)
    if not quizzes:
        print("No quizzes available.")
        return

    # Display all available quizzes grouped by topic
    print("\nAvailable Quizzes by Topic:")
    for quiz_id, quiz in quizzes.items():
        # Check if 'topic' and 'admin' keys exist before accessing them
        topic = quiz.get('topic', 'N/A')  # Use 'N/A' if 'topic' is missing
        admin = quiz.get('admin', 'N/A')  # Use 'N/A' if 'admin' is missing
        print(f"- ID: {quiz_id}, Topic: {topic}, Created by: {admin}")

    # Ask user to choose a quiz by ID
    quiz_id = input("\nEnter the quiz ID you want to take: ").strip()
    if quiz_id not in quizzes:
        print("Quiz not found.")
        return

    # Prompt for the quiz password
    quiz = quizzes[quiz_id]
    quiz_password = quiz["password"]
    password_input = input("Enter the password for this quiz: ")

    if password_input != quiz_password:
        print("Incorrect password. Access denied.")
        return

    # Start the quiz
    score = 0
    print(f"\nStarting quiz '{quiz_id}'...\n")

    for idx, q in enumerate(quiz["questions"], 1):
        print(f"Q{idx}: {q['question']}")
        for i, option in enumerate(q['options'], 1):
            print(f"  {i}. {option}")
        answer = input("Your answer (1-4): ")

        if answer == q['answer']:
            score += 1

    print(f"\nYou completed the quiz with a score: {score}/{len(quiz['questions'])}.")

    # Save the result
    results = load_data(RESULTS_FILE)
    results[username] = results.get(username, {})
    results[username][quiz_id] = {
        "score": score,
        "total_questions": len(quiz['questions']),
        "taken_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    save_data(RESULTS_FILE, results)
    print("Your result has been saved.")

In [11]:
def view_results():
    """
    Loads and displays the quiz results for all participants.
    """
    results = load_data(RESULTS_FILE)
    
    if not results:
        print("No results available.")
        return

    print("\nQuiz Results:")
    for username, user_results in results.items():
        print(f"\nParticipant: {username}")
        for quiz_id, quiz_result in user_results.items():
            score = quiz_result["score"]
            total_questions = quiz_result["total_questions"]
            taken_at = quiz_result["taken_at"]
            print(f"  - Quiz ID: {quiz_id}")
            print(f"    Score: {score}/{total_questions}")
            print(f"    Taken at: {taken_at}")

In [12]:
# Main program loop
def main():
    """
    The main loop of the program that handles user interaction.
    """
    print("Welcome to the Online Quiz System")

    while True:
        print("1. Register")
        print("2. Login")
        print("3. Quit")

        choice = input("Choose an option: ")

        if choice == '1':
            username = input("Enter username: ")
            print(f"you will enter username : {username} ")
            password = input("Enter password: ")
            print("Do you want to see password, please enter 'yes' or 'No'")
            choice=input().lower()
            if choice == 'yes':
                print(f"you enter the password : {password}")
            elif choice == 'no':
                print("I will display password hashed to secure")
                print(f"You entered password: {'*' * len(password)}")

            role = input("Enter role (admin/participant): ").lower()
            register_user(username, password, role)

        elif choice == '2':
            username = input("Enter username: ")
            password = input("Enter password: ")
            role = login_user(username, password)

            if role == 'admin':
                print("Admin Menu")
                while True:
                    print("1. Create Quiz")
                    print("2. View Results")
                    print("3. Logout")

                    admin_choice = input("Choose an option: ")

                    if admin_choice == '1':
                        create_quiz(username)
                    elif admin_choice == '2':
                        view_results()
                    elif admin_choice == '3':
                        break
                    else:
                        print("Invalid choice.")

            elif role == 'participant':
                print("Participant Menu")
                while True:
                    print("1. Take Quiz")
                    print("2. Logout")

                    participant_choice = input("Choose an option: ")

                    if participant_choice == '1':
                        take_quiz(username)
                    elif participant_choice == '2':
                        break
                    else:
                        print("Invalid choice.")
            else:
                print("Login failed.")

        elif choice == '3':
            print("Goodbye!")
            break
        else:
            print("Invalid choice.")

In [ ]:
# Run the main program
main()

Welcome to the Online Quiz System
1. Register
2. Login
3. Quit
you will enter username : Mohamed 
Do you want to see password, please enter 'yes' or 'No'
I will display password hashed to secure
You entered password: ******
Successfully loaded data from users.json
Verification code sent to @gmail.com: 102097
Verification successful!
User 'Mohamed' registered successfully as admin.
1. Register
2. Login
3. Quit
Successfully loaded data from users.json
Welcome, Mohamed!
Admin Menu
1. Create Quiz
2. View Results
3. Logout
Successfully loaded data from results.json

Quiz Results:

Participant: Medo
  - Quiz ID: quiz_1
    Score: 1/1
    Taken at: 2024-10-27 04:39:53
  - Quiz ID: quiz_1630453028
    Score: 1/3
    Taken at: 2024-10-27 04:54:21
1. Create Quiz
2. View Results
3. Logout
Invalid choice.
1. Create Quiz
2. View Results
3. Logout
1. Register
2. Login
3. Quit
Goodbye!
